In [1]:
import numpy as np
from scipy.linalg import logm
import itertools
from time import time

from quantify_core.data.handling import (
    default_datadir,
    get_latest_tuid,
    get_tuids_containing,
    load_dataset,
    locate_experiment_container,
    set_datadir,
)


In [2]:
def compute_transition_matrix(x_list, y_list, counts, j, k):  
    """
    Computes the transition matrix A(j, k) for two-qubit errors based on measurement outcomes.

    Parameters:
    - x_list: List of input bitstrings (each string represents an input quantum state).
    - y_list: List of lists, where each inner list contains possible measured output bitstrings.
    - counts: List of lists, where each inner list gives the count of each corresponding y outcome.
    - j, k: The two qubits of interest.

    Returns:
    - A: The 4x4 transition matrix A(j, k), normalized column-wise.
    """
    
    num_states = 4  # 2-qubit system has 4 possible states: "00", "01", "10", "11"
    A = np.zeros((num_states, num_states))  # Transition matrix
    state_to_index = {"00": 0, "01": 1, "10": 2, "11": 3}
    input_state_counts = np.zeros(num_states)  # Column-wise normalization counts
    
    for x, y_outcomes, y_counts in zip(x_list, y_list, counts):
        input_state = x[j] + x[k]  # Extract 2-qubit input state
        input_index = state_to_index[input_state]  

        # Mask to remove j and k, preserving other qubits
        mask = [i != j and i != k for i in range(len(x))]
        remaining_input_state = "".join([char for char, include in zip(x, mask) if include])

        for y, count in zip(y_outcomes, y_counts):
            measured_state = y[j] + y[k]  # Extract 2-qubit measured state
            remaining_output_state = "".join([char for char, include in zip(y, mask) if include])
            
            if remaining_input_state == remaining_output_state:
                output_index = state_to_index[measured_state]
                A[output_index, input_index] += count
                input_state_counts[input_index] += count  # Track input occurrences
    
    # Normalize each column
    for col in range(num_states):
        if input_state_counts[col] > 0:
            A[:, col] /= input_state_counts[col]

    return A

# Example test case
x_list = ["00", "01", "10", "11"]  # All possible 2-qubit states
y_list = [["00","01"], ["01"], ["10"], ["11"]]  # Transitions
counts = [[1,2], [1], [1], [1]]  # State occurences


j, k = 0, 1  # The two qubits of interest

A_jk = compute_transition_matrix(x_list, y_list, counts, j, k)

G_jk = logm(A_jk)

G_prime_jk = np.copy(G_jk)
G_prime_jk[G_prime_jk<0] = 0

print(A_jk)
# print(G_jk)
# print(G_prime_jk)



[[0.33333333 0.         0.         0.        ]
 [0.66666667 1.         0.         0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         0.         1.        ]]


In [3]:
x_list = ["000", "001", "010", "011", "100", "101", "110", "111"] 
y_list = [
    ["000","001"],  #0_00
    ["001","011"],  
    ["010"],  
    ["011"],  
    ["100"],  
    ["101"],  
    ["110"],  
    ["111"],  
]

counts = [
    [1,1],  
    [1,3],  
    [1],  
    [1],  
    [2],  #1_00
    [1],  
    [1],  
    [1],  
]

j,k = 1,2

test = compute_transition_matrix(x_list, y_list, counts, j, k)

objective = np.sum(test)-np.trace(test)

print(test, objective)

[[0.75 0.   0.   0.  ]
 [0.25 0.4  0.   0.  ]
 [0.   0.   1.   0.  ]
 [0.   0.6  0.   1.  ]] 0.8500000000000001


In [4]:
def objective(x_list, y_list, counts):
    N_qubits = len(x_list[0])
    arr = np.arange(N_qubits)  # [0, 1, 2]
    combinations = list(itertools.combinations(arr, 2))
    
    objective = 0
    
    for i in combinations:
        tmp = compute_transition_matrix(x_list,y_list,counts,i[0],i[0])
        objective += np.sum(tmp) - np.trace(tmp)

    return objective

    

In [5]:
x_list = [
    "0000", "0001", "0010", "0011", 
    "0100", "0101", "0110", "0111", 
    "1000", "1001", "1010", "1011", 
    "1100", "1101", "1110", "1111"
]

y_list = [
    ["0000", "0001"],  # 0_000
    ["0001", "0011"],  
    ["0010", "0110"],  
    ["0011", "0111"],  
    ["0100", "1100"],  
    ["0101"],  
    ["0110"],  
    ["0111"],  
    ["1000"],  
    ["1001"],  
    ["1010"],  
    ["1011"],  
    ["1100"],  
    ["1101"],  
    ["1110"],  
    ["1111"]  
]

counts = [
    [1, 2],  
    [1, 3],  
    [1, 1],  
    [1, 1],  
    [2, 1],  
    [1],  
    [1],  
    [1],  
    [1],  
    [1],  
    [1],  
    [1],  
    [1],  
    [1],  
    [1],  
    [1]  
]


j,k = 1,2

test = compute_transition_matrix(x_list, y_list, counts, j, k)

objective = np.sum(test)-np.trace(test)

print(test, objective)

[[0.57142857 0.         0.         0.        ]
 [0.42857143 0.66666667 0.         0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.33333333 0.         1.        ]] 0.7619047619047619


In [6]:
x_list = [
    "0000", "0001", "0010", "0011", 
    "0100", "0101", "0110", "0111", 
    "1000", "1001", "1010", "1011", 
    "1100", "1101", "1110", "1111"
]

y_list = [
    ["0000", "0001", "0010", "0100"],  # 0000 -> multiple states
    ["0001", "0011", "0111"],  
    ["0010", "0110", "1010"],  
    ["0011", "0111", "1011"],  
    ["0100", "1100", "0101", "0000"],  
    ["0101", "0001", "1001"],  
    ["0110", "1010", "1110"],  
    ["0111", "1011", "1111", "0110"],  
    ["1000", "0000", "1100", "1110"],  
    ["1001", "0101", "1101"],  
    ["1010", "0110", "1110", "1011"],  
    ["1011", "0011", "0111", "1111"],  
    ["1100", "0100", "1000", "1110"],  
    ["1101", "1001", "1111"],  
    ["1110", "0110", "1010", "1100"],  
    ["1111", "0111", "1011", "1101"]  
]

counts = [
    [1, 2, 1, 3],  
    [1, 3, 2],  
    [2, 1, 1],  
    [1, 2, 1],  
    [2, 1, 1, 3],  
    [1, 2, 1],  
    [2, 1, 1],  
    [1, 2, 1, 3],  
    [2, 3, 1, 1],  
    [1, 2, 1],  
    [1, 2, 3, 1],  
    [1, 3, 2, 1],  
    [2, 1, 1, 3],  
    [1, 2, 1],  
    [2, 3, 1, 1],  
    [1, 2, 1, 3]  
]

j,k = 1,2

start = time()

test = compute_transition_matrix(x_list, y_list, counts, j, k)

end = time()

objective = np.sum(test)-np.trace(test)

print(test, objective)

print("dt:", (end-start))

[[0.29411765 0.         0.44444444 0.        ]
 [0.23529412 0.41666667 0.         0.16666667]
 [0.29411765 0.         0.33333333 0.33333333]
 [0.17647059 0.58333333 0.22222222 0.5       ]] 2.4558823529411757
dt: 0.00016379356384277344


In [12]:
def hadamard_calibration(n):
    p = int(np.ceil(np.log2(n)))  # Smallest p such that n < 2^p
    C = []
    
    for a in range(2**p):
        a_bin = [int(b) for b in format(a, f'0{p}b')]  # Convert a to binary list of length p
        x_a = ""
        
        for b in range(1, n + 1):
            b_bin = [int(b) for b in format(b, f'0{p}b')]  # Convert b to binary list of length p
            bit = sum(ai * bi for ai, bi in zip(a_bin, b_bin)) % 2  # Compute x_b^a (mod 2)
            x_a += str(bit)
        
        C.append(x_a)
    
    return C

print(hadamard_calibration(17))
# print(len(hadamard_calibration(17)))

# 32*10000*(200*1e-6)

# # Example usage:
# n = 4
# bitstrings = hadamard_calibration(n)
# print(bitstrings)

# x_list = bitstrings

# y_list = [
#     ["0001", "0010", "0100", "1000", "1111"],  # Diverse transitions
#     ["0000", "0011", "0110", "1001", "1110", "1101"],  
#     ["0010", "0101", "0110", "1011", "1100"],  
#     ["0000", "0011", "1001", "1010", "1111"]  
# ]

# counts = [
#     [2, 1, 3, 1, 2],  
#     [1, 3, 2, 1, 2, 1],  
#     [3, 1, 2, 1, 3],  
#     [2, 1, 3, 1, 2]  
# ]

# j,k = 1,2

# start = time()

# test = compute_transition_matrix(x_list, y_list, counts, j, k)

# end = time()

# objective = np.sum(test)-np.trace(test)

# print(test, objective)

# print("dt:", (end-start))



['00000000000000000', '10101010101010101', '01100110011001100', '11001100110011001', '00011110000111100', '10110100101101001', '01111000011110000', '11010010110100101', '00000001111111100', '10101011010101001', '01100111100110000', '11001101001100101', '00011111111000000', '10110101010010101', '01111001100001100', '11010011001011001', '00000000000000011', '10101010101010110', '01100110011001111', '11001100110011010', '00011110000111111', '10110100101101010', '01111000011110011', '11010010110100110', '00000001111111111', '10101011010101010', '01100111100110011', '11001101001100110', '00011111111000011', '10110101010010110', '01111001100001111', '11010011001011010']


In [8]:
def compute_transition_matrix_dict(data, j, k):  
    """
    Computes the transition matrix A(j, k) for two-qubit errors based on measurement outcomes.

    Parameters:
    - data: Dictionary where keys are input bitstrings and values are dictionaries.
      Each inner dictionary has measured output bitstrings as keys and their counts as values.
    - j, k: The two qubits of interest.

    Returns:
    - A: The 4x4 transition matrix A(j, k), normalized column-wise.
    """
    num_states = 4  # 2-qubit system has 4 possible states: "00", "01", "10", "11"
    A = np.zeros((num_states, num_states))  # Transition matrix
    state_to_index = {"00": 0, "01": 1, "10": 2, "11": 3}
    input_state_counts = np.zeros(num_states)  # Column-wise normalization counts
    
    for x, outcomes in data.items():
        input_state = x[j] + x[k]  # Extract 2-qubit input state
        input_index = state_to_index[input_state]  

        # Mask to remove j and k, preserving other qubits
        mask = [i != j and i != k for i in range(len(x))]
        remaining_input_state = "".join([char for char, include in zip(x, mask) if include])

        for y, count in outcomes.items():
            measured_state = y[j] + y[k]  # Extract 2-qubit measured state
            remaining_output_state = "".join([char for char, include in zip(y, mask) if include])
            
            if remaining_input_state == remaining_output_state:
                output_index = state_to_index[measured_state]
                A[output_index, input_index] += count
                input_state_counts[input_index] += count  # Track input occurrences
    
    # Normalize each column
    for col in range(num_states):
        if input_state_counts[col] > 0:
            A[:, col] /= input_state_counts[col]

    return A

# Example test case
data = {
    "00": {"00": 1, "01": 2},
    "01": {"01": 1},
    "10": {"10": 1},
    "11": {"11": 1}
}

j, k = 0, 1  # The two qubits of interest
A_jk = compute_transition_matrix_dict(data, j, k)
print(A_jk)

[[0.33333333 0.         0.         0.        ]
 [0.66666667 1.         0.         0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         0.         1.        ]]


In [9]:
#Extract attributes: dataset.dataset.attrs (get x0 and number_of_repetitions)
empty_dict = {}
empty_dict2 = {}

empty_dict2["11"] = 1
empty_dict2["01"] = 2

empty_dict2["01"] += 1

empty_dict["00"] = empty_dict2

print(empty_dict)

{'00': {'11': 1, '01': 3}}


In [10]:
reps = 4
x0 = ["0000", "1111"]
N_states = len(x0)

y0 = np.array([0, 1, 1, 0, 1, 0, 1, 1])
y1 = np.array([0, 0, 1, 0, 1, 0, 1, 1])
y2 = np.array([0, 0, 1, 0, 1, 0, 1, 1])
y3 = np.array([0, 0, 1, 0, 1, 0, 1, 1])  # Slight variation for diversity

#Quantify core function: to_gridded_dataset

def measured_to_counts(dataset):

    num_elements = len(dataset.attrs["elements"])
    stacked = np.column_stack([dataset[f"y{i}"].values for i in range (num_elements) #Something like this
    # Stack the arrays and then concatenate column-wise
    stacked = np.column_stack([y0, y1, y2, y3])
    stacked_strings = np.array(["".join(row.astype(str)) for row in stacked])
    
    # Reshape into groups of `reps`
    reshaped_strings = stacked_strings.reshape(N_states, reps)
    
    # Use NumPy’s optimized unique function to count occurrences
    result = {x0[i]: dict(zip(*np.unique(reshaped_strings[i], return_counts=True))) for i in range(N_states)}
    
    return result


SyntaxError: '[' was never closed (1099100881.py, line 15)